In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
import cv2
import torch
import datetime
import requests
from ultralytics import solutions

# Optional: Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')

# Settings
video_path = "/content/drive/MyDrive/src_files/IMG_2429_v2.MOV"  # Change if not using Kaggle
model_path = "/content/drive/MyDrive/src_files/pothole_yolo.pt"
output_dir = '/content/drive/MyDrive/Results'

os.makedirs(output_dir, exist_ok=True)

# Your Telegram bot token and chat ID
BOT_TOKEN = '7493712820:AAGFA9D53gHjaZ_qT7YEcDq_6DKKLpo2FW0'
CHAT_ID = '6021660418'

def send_telegram_message(message):
    url = f'https://api.telegram.org/bot{BOT_TOKEN}/sendMessage'
    payload = {'chat_id': CHAT_ID, 'text': message}
    try:
        requests.post(url, data=payload)
    except Exception as e:
        print("Failed to send Telegram message:", e)

# Torch Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load Video
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

# Video properties
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Object Counter
region_points = [(200, 800), (1900, 800)]
counter = solutions.ObjectCounter(
    show=False,
    region=region_points,
    model=model_path,
    device=device
)

# Start from specific frame
START_FRAME = 0
cap.set(cv2.CAP_PROP_POS_FRAMES, START_FRAME)

# Frame indexing
frame_idx = START_FRAME
chunk_size = 300
chunk_number = START_FRAME // chunk_size
frame_idx_in_chunk = START_FRAME % chunk_size

# Video writer & log file
video_file_name = f"object_counting_output_{chunk_number*chunk_size}_{(chunk_number+1)*chunk_size - 1}.avi"
log_file_name = f"log_chunk_{chunk_number*chunk_size}_{(chunk_number+1)*chunk_size - 1}.txt"

video_writer = cv2.VideoWriter(video_file_name, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
log_file = open(log_file_name, "a")

send_telegram_message(f"🚀 Started processing from frame {START_FRAME}")

track = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        log_file.write("Video frame is empty or processing is complete.\n")
        break

    if frame_idx >= 99601:
      break

    # Process frame
    results = counter(frame)
    video_writer.write(frame)

    log_file.write("========\n")
    log_file.write(str(results) + "\n")
    log_file.write(f"Processing frame #{frame_idx}\n")
    log_file.write("========\n")
    log_file.flush()

    print(f"Processing frame #{frame_idx}\n")

    if frame_idx_in_chunk % 1000 == 0:
        send_telegram_message(f"✅ Processed {frame_idx_in_chunk} frames in chunk {chunk_number + 1}")

    frame_idx += 1
    frame_idx_in_chunk += 1

    # If chunk ends
    if frame_idx % chunk_size == 0:
        video_writer.release()
        log_file.close()

        # Convert AVI to MP4
        mp4_file = video_file_name.replace(".avi", ".mp4")
        os.system(f"ffmpeg -y -i {video_file_name} -vcodec libx264 -acodec aac {mp4_file}")

        # Move to Drive
        import shutil
        shutil.move(mp4_file, os.path.join(output_dir, mp4_file))
        shutil.move(log_file_name, os.path.join(output_dir, log_file_name))

        print("===================")
        print(f"Video file name:{video_file_name}")
        print("===================")

        os.remove(video_file_name)  # Delete the .avi fil

        send_telegram_message(f"📤 Uploaded: {mp4_file}")

        # Next chunk
        chunk_number += 1
        frame_idx_in_chunk = 0

        # New file names
        video_file_name = f"object_counting_output_{chunk_number*chunk_size}_{(chunk_number+1)*chunk_size - 1}.avi"
        log_file_name = f"log_chunk_{chunk_number*chunk_size}_{(chunk_number+1)*chunk_size - 1}.txt"

        video_writer = cv2.VideoWriter(video_file_name, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
        log_file = open(log_file_name, "a")

# Final cleanup
cap.release()
video_writer.release()
log_file.close()
# cv2.destroyAllWindows()
send_telegram_message("✅ All processing complete!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cpu
Ultralytics Solutions: ✅ {'source': None, 'model': '/content/drive/MyDrive/src_files/pothole_yolo.pt', 'classes': None, 'show_conf': True, 'show_labels': True, 'region': [(200, 800), (1900, 800)], 'colormap': 21, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': False, 'iou': 0.7, 'conf': 0.25, 'device': 'cpu', 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': True, 'data': 'images'}
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


0: 384x640 (no detections), 201.8ms
Speed: 3.5ms preprocess, 201.8ms inf

In [4]:
import os
import re

# # Optional: Mount Google Drive (for Colab)
# from google.colab import drive
# drive.mount('/content/drive')

# Set your folder path here
folder_path = "/content/drive/MyDrive/Results"

# Expected chunk ranges
expected_ranges = [f"{i}_{i+299}" for i in range(49800, 99600, 300)]

# File lists
txt_chunks = set()
mp4_chunks = set()

# List all files and extract ranges
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        match = re.search(r"log_chunk_(\d+_\d+)\.txt", filename)
        if match:
            txt_chunks.add(match.group(1))
    elif filename.endswith(".mp4"):
        match = re.search(r"object_counting_output_(\d+_\d+)\.mp4", filename)
        if match:
            mp4_chunks.add(match.group(1))

# Find missing ranges
missing_txt = sorted(set(expected_ranges) - txt_chunks)
missing_mp4 = sorted(set(expected_ranges) - mp4_chunks)

print("📄 Missing TXT chunks:", missing_txt if missing_txt else "None")
print("🎞️ Missing MP4 chunks:", missing_mp4 if missing_mp4 else "None")

📄 Missing TXT chunks: ['99300_99599']
🎞️ Missing MP4 chunks: ['99300_99599']
